In [13]:
from gensim.models import CoherenceModel
from gensim.corpora import Dictionary
from gensim.models import LdaModel
import pandas as pd
import gensim
from gensim import corpora
from nltk.corpus import stopwords
import spacy


data = pd.read_csv("data.csv")
print(data['content'].info())
dataFiltered = data["content"].drop_duplicates().dropna()[:35000]

stop_words = stopwords.words('english')

# function to remove stopwords
def remove_stopwords(text):
    textArr = text.split(' ')
    rem_text = " ".join([i for i in textArr if i not in stop_words])
    return rem_text

# remove stopwords from the text
dataFiltered=dataFiltered.apply(remove_stopwords)

# nlp = en_core_web_sm.load()
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

def lemmatization(texts,allowed_postags=['NOUN', 'ADJ']): 
       output = []
       for sent in texts:
             doc = nlp(sent) 
             output.append([token.lemma_ for token in doc if token.pos_ in allowed_postags ])
       return output

text_list=dataFiltered.tolist()
#print(text_list[5])
tokenized_reviews = lemmatization(text_list)
#print(tokenized_reviews[5])

dictionary = corpora.Dictionary(tokenized_reviews)

doc_term_matrix = [dictionary.doc2bow(doc) for doc in tokenized_reviews]

# Creating the object for LDA model using gensim library
Lda = gensim.models.ldamodel.LdaModel

# Running and Trainign LDA model on the document term matrix.
ldamodel = Lda(corpus=doc_term_matrix, id2word=dictionary, num_topics=20, random_state=100, chunksize=100, passes=10)
print("-"*10, "\n", "Topics\n", ldamodel.print_topics(num_words=10))



<class 'pandas.core.series.Series'>
RangeIndex: 50000 entries, 0 to 49999
Series name: content
Non-Null Count  Dtype 
--------------  ----- 
50000 non-null  object
dtypes: object(1)
memory usage: 390.8+ KB
None


c:\Python310\lib\site-packages\catalogue\__init__.py:135: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):
c:\Python310\lib\site-packages\catalogue\__init__.py:135: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):
c:\Python310\lib\site-packages\catalogue\__init__.py:135: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):
c:\Python310\lib\site-packages\catalogue\__init__.py:135: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):
c:\Python310\lib\site-packages\catalogue\__init__.py:123: DeprecationWarning: SelectableGroups dict interface is deprecated. Use

---------- 
 Topics
 [(0, '0.066*"company" + 0.034*"money" + 0.028*"business" + 0.014*"firm" + 0.013*"bank" + 0.013*"sale" + 0.012*"employee" + 0.011*"dollar" + 0.011*"fund" + 0.011*"financial"'), (1, '0.090*"student" + 0.089*"school" + 0.052*"black" + 0.048*"white" + 0.035*"community" + 0.026*"group" + 0.022*"event" + 0.021*"university" + 0.020*"education" + 0.020*"college"'), (2, '0.049*"police" + 0.031*"officer" + 0.025*"people" + 0.020*"death" + 0.016*"man" + 0.015*"incident" + 0.013*"car" + 0.012*"authority" + 0.012*"video" + 0.011*"murder"'), (3, '0.027*"study" + 0.024*"health" + 0.023*"drug" + 0.019*"medical" + 0.018*"disease" + 0.018*"case" + 0.016*"research" + 0.016*"patient" + 0.014*"doctor" + 0.012*"scientist"'), (4, '0.028*"court" + 0.027*"law" + 0.027*"order" + 0.024*"case" + 0.022*"federal" + 0.020*"attorney" + 0.014*"executive" + 0.014*"legal" + 0.014*"judge" + 0.013*"state"'), (5, '0.099*"vote" + 0.072*"voter" + 0.070*"election" + 0.043*"party" + 0.040*"poll" + 0.034*"s

In [14]:
import pyLDAvis
import pyLDAvis.gensim_models
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(ldamodel, doc_term_matrix, dictionary)
vis


C:\Users\eslam\AppData\Roaming\Python\Python310\site-packages\pyLDAvis\_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  by='saliency', ascending=False).head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
7     -0.060036  0.066666       1        1  14.071278
13    -0.156453 -0.018811       2        1  12.560491
15    -0.114175  0.016612       3        1   9.541844
18    -0.096096  0.089036       4        1   6.728816
10     0.013091  0.165421       5        1   6.394216
11    -0.133334  0.092223       6        1   5.862175
4     -0.154129  0.024806       7        1   5.722170
2      0.000899  0.232444       8        1   5.486210
12    -0.042918 -0.197505       9        1   4.623000
6     -0.178905 -0.018725      10        1   4.615996
0      0.001960 -0.127587      11        1   3.945521
19    -0.095920 -0.123970      12        1   3.186232
17     0.014747 -0.002077      13        1   3.078541
5      0.056950 -0.208734      14        1   2.688705
14     0.211475  0.154824      15        1   2.581901
3     -0.038819 -0.053538      16        1   2.490286
1      0.079648  0.054618      17        1   2.171723
9      0.066031 -0.116834      18        1   2.045504
16     0.272086  0.092044      19        1   1.659510
8      0.353898 -0.120915      20        1   0.545882, topic_info=           Term          Freq         Total Category  logprob  loglift
47     election  23514.000000  23514.000000  Default  30.0000  30.0000
1875       vote  18125.000000  18125.000000  Default  29.0000  29.0000
437          m.  10827.000000  10827.000000  Default  28.0000  28.0000
696       woman  20598.000000  20598.000000  Default  27.0000  27.0000
762     company  15650.000000  15650.000000  Default  26.0000  26.0000
...         ...           ...           ...      ...      ...      ...
704          ’s     48.353182  14876.035576  Topic20  -6.5185  -0.5185
7219    tonight      0.122562      1.207839  Topic20 -12.4962   2.9226
6387     county      0.122562      1.207854  Topic20 -12.4962   2.9225
7056    weather      0.122562      1.207857  Topic20 -12.4962   2.9225
11024         %      0.122566      1.208473  Topic20 -12.4962   2.9221

[1275 rows x 6 columns], token_table=      Topic      Freq Term
term                      
1879      1  0.043487    .
1879      4  0.017477    .
1879      6  0.003578    .
1879      9  0.380788    .
1879     11  0.138581    .
...     ...       ...  ...
704      10  0.001815   ’s
704      12  0.015797   ’s
704      14  0.008134   ’s
704      18  0.010419   ’s
704      20  0.003227   ’s

[3183 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[8, 14, 16, 19, 11, 12, 5, 3, 13, 7, 1, 20, 18, 6, 15, 4, 2, 10, 17, 9])

In [15]:
print('\nPerplexity: ', ldamodel.log_perplexity(doc_term_matrix,total_docs=10000))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
from gensim.models.coherencemodel import CoherenceModel
coherence_model_lda = CoherenceModel(model=ldamodel, texts=tokenized_reviews, dictionary=dictionary , coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -10.616825852472388

Coherence Score:  0.5206197133123643


In [16]:
# Get topic weights and dominant topics ------------
from sklearn.manifold import TSNE
from bokeh.plotting import figure, output_file, show
from bokeh.models import Label
from bokeh.io import output_notebook
import numpy as np  

# Get topic weights
topic_weights = []
for i, row_list in enumerate(ldamodel[doc_term_matrix]):
    topic_weights.append([w for i, w in row_list[0]])

# Array of topic weights    
arr = pd.DataFrame(topic_weights).fillna(0).values

# Keep the well separated points (optional)
arr = arr[np.amax(arr, axis=1) > 0.35]

# Dominant topic number in each doc
topic_num = np.argmax(arr, axis=1)

# tSNE Dimension Reduction
tsne_model = TSNE(n_components=2, verbose=1, random_state=0, angle=.99, init='pca')
tsne_lda = tsne_model.fit_transform(arr)

# Plot the Topic Clusters using Bokeh
output_notebook()
n_topics = 4
mycolors = np.array([color for name, color in mcolors.TABLEAU_COLORS.items()])
plot = figure(title="t-SNE Clustering of {} LDA Topics".format(n_topics), 
              plot_width=900, plot_height=700)
plot.scatter(x=tsne_lda[:,0], y=tsne_lda[:,1], color=mycolors[topic_num])
show(plot)

TypeError: cannot unpack non-iterable int object